### Icevision Inference and Evalutation

This notebook walks through the steps to load a portable torchscript model with torch and run inference on our validation set (or test set)

Loading a Torchscript scripting model.

Reminder: if this cell is failing, remember to mount the GCP buckets with `cdata` and `cdata2`

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
import random
from pycocotools.cocoeval import COCOeval
from ceruleanml.coco_stats import all_sample_stat_lists
from ceruleanml.coco_load_fastai import record_collection_to_record_ids, get_image_path, record_to_mask
from ceruleanml import preprocess
from ceruleanml import data
from icevision import models, parsers, show_records, tfms, Dataset, Metric, COCOMetric, COCOMetricType, show_records
import numpy as np
import skimage.io as skio
from ceruleanml.inference import keep_by_global_pixel_nms, keep_boxes_by_idx, keep_by_bbox_score, flatten_pixel_segmentation, flatten_gt_segmentation
from ceruleanml import data
from pathlib import Path

import torch
import torchvision #Torchvision is required to load the model, it does a global import in the background to include the NMS operation that Mask-R-CNN needs.
print(torchvision.__version__)
print(torch.__version__)

In [ ]:
from ceruleanml.learner_config import (
    run_list,
    classes_to_keep,
    get_tfms,
    wd,
    record_collection_train,
    record_collection_val,
    record_collection_test,
    # record_collection_rrctrained,
    thresholds,
)

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [ ]:
# icevision_experiment_dir = Path("/root/experiments/cv2/2023_05_22_15_51_39_4cls_rn34_pr224_px1024_60min_maskrcnn") # infra+density
icevision_experiment_dir = Path("/root/experiments/cv2/2023_06_03_01_20_11_4cls_rn34_pr224_px1024_60min_maskrcnn") # infra+density3
# icevision_experiment_dir = Path("/root/experiments/cv2/2023_06_03_23_25_07_4cls_rn34_pr224_px1024_60min_maskrcnn") # infra+density4
# icevision_experiment_dir = Path("/root/experiments/cv2/2023_06_04_15_36_57_4cls_rn34_pr224_px1024_60min_maskrcnn") # infra+density5
# icevision_experiment_dir = Path("/root/experiments/cv2/2023_06_05_13_04_01_4cls_rn34_pr224_px1024_60min_maskrcnn") # triplicate2
# icevision_experiment_dir = Path("/root/experiments/cv2/2023_06_07_03_59_53_4cls_rn34_pr224_px1024_60min_maskrcnn") # infra+density6

# icevision_experiment_dir = Path("/root/experiments/cv2/2023_06_08_04_26_22_4cls_rn34_pr224_px1024_360min_maskrcnn") # 360 infra+density
icevision_experiment_dir = Path("/root/experiments/cv2/2023_06_23_10_52_52_4cls_rn34_pr512_px1024_120min_maskrcnn") # 512
icevision_experiment_dir = Path("/root/experiments/cv2/2023_09_26_18_30_13_4cls_rnxt101_pr512_px1024_600min_maskrcnn") # DEPLOYED
# icevision_experiment_dir = Path("/root/experiments/cv2/2023_09_27_19_30_30_2cls_rnxt101_pr512_px1024_240min_maskrcnn") # 512
# icevision_experiment_dir = Path("/root/experiments/cv2/2023_09_28_10_12_53_4cls_rnxt101_pr1024_px1024_240min_maskrcnn") # 512
# icevision_experiment_dir = Path("/root/experiments/cv2/2023_09_29_15_03_32_4cls_rnxt101_pr512_px1024_240min_maskrcnn_roix2_intermediate") # 512
icevision_experiment_dir = Path("/root/experiments/cv2/2023_09_30_02_50_25_4cls_rnxt101_pr512_px1024_240min_intermediate_maskrcnn_roix8") # 512

scripted_model = torch.jit.load(f"{icevision_experiment_dir}/scripting_cpu_model.pt", map_location="cpu")

Setting up our Icevision record collection for the evaluation dataset.

In [ ]:
 # Add a number after the colon to reduce the evaluation dataset size for faster eval
eval_record_collection = record_collection_test[:] # record_collection_test, record_collection_val, record_collection_rrctrained
eval_ds = Dataset(eval_record_collection, get_tfms()[1])

Grabbing a test image to inspect

In [ ]:
# nrows, ncols = 4, 2
# rand_start = random.randint(0,len(eval_ds)-ncols*nrows)
# show_records(eval_record_collection[rand_start:rand_start+ncols*nrows], ncols=ncols, class_map=classes_to_keep, display_mask=True, display_bbox=False)
# print("\n".join([str(np.arange(rand_start+ncols*i,rand_start+ncols*(i+1))) for i in np.arange(nrows)]))

In [ ]:
eval_idx = 4

eval_record = eval_ds[eval_idx]
eval_img = eval_record.img
cats = eval_record.detection.labels

if cats:
    mask_sum = np.sum([np.multiply(classes_to_keep.index(cat),eval_record.detection.mask_array[i].data) for i, cat in enumerate(cats)],0)
else:
    mask_sum = np.zeros([1,run_list[-1][0],run_list[-1][0]])
# show_records([eval_record],class_map=classes_to_keep, display_mask=True, display_bbox=False)
skio.imshow_collection([eval_img[:,:,0],mask_sum[0,:,:]], interpolation="nearest")
print(f"Ground Truth classes: {cats}")


In [ ]:
type(eval_img)
# We need to convert this to a pytorch tensor before running inference with the model we loaded
torch.Tensor(np.moveaxis(eval_img,2,0)).shape
# And normalize the values to fall between 0 and 1
[torch.Tensor(np.moveaxis(eval_img,2,0))/255]
# Finally, we need to put each sample tensor in a list, the list length is the batch dimension.
len([torch.Tensor(np.moveaxis(eval_img,2,0))/255])

# maskrcnn wants a list of 3D arrays with length of list as batch size, Fastai Unet wants a 4D array with 0th dim as batch size
losses, pred_list = scripted_model([torch.Tensor(np.moveaxis(eval_img,2,0))/255])

# fastai returns a 2D array of logits. logits need to be converted to confidence probabilities. 
# Mask RCNN returns a losses array we don't use and a list of dictionaries containing detections.
len(pred_list)
len(pred_list[0]["labels"])

# We can extract the first mask in the first sample's prediction and plot it by converting it to a numpy array.
# skio.imshow(pred_list[0]['masks'][0,0,:,:].detach().cpu().numpy())
#TODO plot a histogram of confidence scores for a pred_list

# pred_list details
# * bbox coords are not normalized. 
# * dict values are tensors until post processed with conf thresholding.
# * length of value list indicates how many instances detected both low and high confidence
# * Mask R-CNN mask values are not logits, they are 0 to 1 confidence probabilities. the torchscript model applies softmax unlike the fastai unet model where we do that after inference.
# * bbox coord order is xmin, ymin, xmax, ymax, the same as icevision record collection bbox format

# pred_list[0]

# # scratch code
# import matplotlib.pyplot as plt
# test = pred_list[0]['masks'][1,0,:,:].detach().cpu().numpy().flatten()
# test[test == 0] = np.nan
# plt.hist(test, bins = 300,log=True)

After inference, we need to post process the result.

In [ ]:
pred_dict = pred_list[0] # We only ran one image through inference
print(f"\nPredicted classes before thresholds:\n { {round(pred_list[0]['scores'][i].item(),2): classes_to_keep[l] for i,l in enumerate(pred_list[0]['labels'])}}\n")

bbox_score_thresh = 0. or thresholds["bbox_score_thresh"]
keep = keep_by_bbox_score(pred_dict, bbox_score_thresh)
pred_dict = keep_boxes_by_idx(pred_dict, keep)

print(f"Predicted classes after bbox_score_thresh={bbox_score_thresh}:\n { {round(pred_dict['scores'][i].item(),2):classes_to_keep[l] for i, l in enumerate(pred_dict['labels'])} }\n")

pixel_nms_thresh = 0. or thresholds["pixel_nms_thresh"]
keep = keep_by_global_pixel_nms(pred_dict, pixel_nms_thresh)
pred_dict = keep_boxes_by_idx(pred_dict, keep)

print(f"Predicted classes after pixel_nms_thresh={pixel_nms_thresh}:\n { {round(pred_dict['scores'][i].item(),2):classes_to_keep[l] for i, l in enumerate(pred_dict['labels'])} }\n")

skio.imshow_collection([eval_img[:,:,0], *[m[0,:,:].detach().cpu().numpy() for m in pred_dict['masks']]], interpolation="nearest")

poly_score_thresh = 0. or thresholds["poly_score_thresh"]
semantic_mask_pred = flatten_pixel_segmentation([pred_dict], poly_score_thresh, eval_img.shape[:2])[0]
# The output of the last thresholding step is a 2D array of classes. we use this for pixel-wise evaluation. 

skio.imshow_collection([eval_img[:,:,0],semantic_mask_pred, mask_sum[0,:,:]], interpolation="nearest")
print(f"Classes after all thresholds:\n { {int(o):classes_to_keep[o] for o in np.unique(semantic_mask_pred)} }")

print(f"\nGround Truth classes: {cats}")


In [ ]:
import matplotlib.pyplot as plt

# Custom imshow_collection function with labels
def custom_imshow_collection_with_labels(image_collection, labels):
    fig, axes = plt.subplots(1, 3, figsize=(12,4))  # 1 row, 3 columns
    for i, ax in enumerate(axes):
        if i < len(image_collection):
            ax.imshow(image_collection[i], interpolation="nearest")
            ax.axis('off')
            ax.set_title(labels[i])
        else:
            ax.axis('off')
    plt.show()

# Labels for the images
labels = ["VV", "Ground Truth", "Prediction"]

# Display the images with labels
custom_imshow_collection_with_labels([eval_img[:,:,0], mask_sum[0,:,:], semantic_mask_pred], labels)


# Confusion Matrices

In [ ]:
from ceruleanml.learner_config import thresholds
from ceruleanml.inference import raw_predict, reduce_preds

raw_preds = raw_predict(scripted_model, eval_ds)

reduced_preds = reduce_preds(
    raw_preds,
    bbox_score_thresh = thresholds["bbox_score_thresh"], 
    pixel_score_thresh = thresholds["pixel_score_thresh"],
    pixel_nms_thresh = thresholds["pixel_nms_thresh"], 
    poly_score_thresh = thresholds["poly_score_thresh"], 
    )

In [ ]:
# PIXEL CM
from ceruleanml.inference import flatten_gt_segmentation, flatten_pixel_segmentation
from ceruleanml.evaluation import cm_f1

flat_gts = flatten_gt_segmentation(eval_ds)
flat_preds = flatten_pixel_segmentation(reduced_preds, poly_score_thresh = thresholds["poly_score_thresh"], shape=eval_ds[0].common.img.shape[:2])

cm, f1 = cm_f1(
    flat_gts,
    flat_preds,
    save_dir=icevision_experiment_dir, 
    normalize=None, 
    class_names=eval_ds[0].detection.class_map.get_classes(),
    title=f"Pixel Confusion Matrix Mask R-CNN: {icevision_experiment_dir}"
)
for row in cm:
    print(', '.join([str(v) for v in row]))

In [ ]:
# INSTANCE CM
from icevision.metrics import SimpleConfusionMatrix
# from ceruleanml.learner_config import thresholds
from ceruleanml.evaluation import gt_pixel_dice, gt_bbox_dice

cm = SimpleConfusionMatrix(
    groundtruth_dice_thresh = thresholds["groundtruth_dice_thresh"],
    groundtruth_dice_function = gt_pixel_dice) # this class is edited on SkyTruth fork
cm.accumulate(eval_ds, reduced_preds)
cm.finalize()
for row in cm.confusion_matrix:
    print(', '.join([str(v) for v in row]))
    
cm.plot(figsize=5, normalize=None)

# Upload model to Bucket for Deployment

In [ ]:
import subprocess
# subprocess.run(f"gsutil -m cp -r {icevision_experiment_dir} gs://ceruleanml/experiments/", shell=True)

In [ ]:
# Run this SQL manually to update the database
type_var = "MASKRCNN"
layers_var = ["VV","INFRA","VESSEL"]
cls_map_var = {0: "BACKGROUND", 1: "INFRA", 2: "NATURAL", 3: "VESSEL"}
name_var = "ResNext 101"
tile_width_m_var = 20422
tile_width_px_var = 512
epochs_var = 122
backbone_size_var = 101
pixel_f1_var = 0.434
instance_f1_var = 0.414
sql = f"""
INSERT INTO model (type, file_path, layers, cls_map, name, tile_width_m, tile_width_px, epochs, thresholds, backbone_size, pixel_f1, instance_f1)
VALUES ('{type_var}', 'experiments/{icevision_experiment_dir.name}/scripting_cpu_model.pt', ARRAY{layers_var}, '{cls_map_var}'::json, '{name_var}', {tile_width_m_var}, {tile_width_px_var}, {epochs_var}, '{thresholds}'::json, {backbone_size_var}, {pixel_f1_var}, {instance_f1_var});
"""
print(sql)


In [ ]:
# Search for and update:
#   cerulean-cloud-images:weigths_name: experiments/2023_09_26_18_30_13_4cls_rnxt101_pr512_px1024_600min_maskrcnn/scripting_cpu_model.pt


# Fastai Inference Evaluation with a Pixel Wise confusion matrix

Like with icevision, we can run inference with a portable torchscript model. 

In [ ]:
fastai_unet_experiment_dir = "/root/data/experiments/cv2/29_Jun_2022_06_36_38_fastai_unet"

tracing_model_cpu_pth = f"{fastai_unet_experiment_dir}/tracing_cpu_224_120__512_36__4_34_0.0003_0.436.pt"

model = torch.jit.load(tracing_model_cpu_pth)

Next, we set up the data loader.

In [ ]:
from ceruleanml import data
from ceruleanml import evaluation
from ceruleanml import preprocess
import os, random
import skimage.io as skio
import numpy as np
from ceruleanml.coco_load_fastai import record_collection_to_record_ids, get_image_path, record_to_mask
from torchvision.models import resnet18, resnet34, resnet50
from fastai.data.block import DataBlock
from fastai.vision.data import ImageBlock, MaskBlock
from fastai.data.transforms import IndexSplitter
from fastai.vision.augment import aug_transforms, Resize
from ceruleanml.inference import logits_to_classes, apply_conf_threshold

bs_d ={512:4, 256:32, 224:32, 128:64, 64:256}
lr_d = {512:3e-4, 256:1e-3, 224:3e-3, 128:3e-3, 64:1e-2}
arch_d = {18: resnet18, 34: resnet34, 50: resnet50}
class_model_file = "/root/experiments/cv2/26_Jul_2022_21_57_24_fastai_unet/tracing_cpu_test_32_34_224_0.824_30.pt"

### Parsing COCO Dataset with Icevision

# for fastai we need the train set to parse the val set with fastai dls
mount_path = "/root/"
train_set = "train-with-context-512"
tiled_images_folder_train = "tiled_images"
json_name_train = "instances_TiledCeruleanDatasetV2.json"

classes_to_remove=[
    "ambiguous",
    ]
classes_to_remap ={
    # "old_vessel": "recent_vessel",
    # "coincident_vessel": "recent_vessel",
}

coco_json_path_train = f"{mount_path}/partitions/{train_set}/{json_name_train}"
tiled_images_folder_train = f"{mount_path}/partitions/{train_set}/{tiled_images_folder_train}"
record_collection_with_negative_small_filtered_train = preprocess.load_set_record_collection(
    coco_json_path_train, tiled_images_folder_train, area_thresh, negative_sample_count_val, preprocess=False, 
    classes_to_remap=classes_to_remap, classes_to_remove=classes_to_remove
)
record_collection_with_negative_small_filtered_val = preprocess.load_set_record_collection(
    coco_json_path_val, tiled_images_folder_val, area_thresh, negative_sample_count_val, preprocess=True,
    classes_to_remap=classes_to_remap, classes_to_remove=classes_to_remove
)

record_ids_train = record_collection_to_record_ids(record_collection_with_negative_small_filtered_train)

record_ids_val = record_collection_to_record_ids(record_collection_with_negative_small_filtered_val)

assert len(set(record_ids_train)) + len(set(record_ids_val)) == len(record_ids_train) + len(record_ids_val)

train_val_record_ids = record_ids_train + record_ids_val
combined_record_collection = record_collection_with_negative_small_filtered_train + record_collection_with_negative_small_filtered_val

def get_val_indices(combined_ids, val_ids):
    return list(range(len(combined_ids)))[-len(val_ids):]

### Constructing a FastAI DataBlock that uses parsed COCO Dataset from icevision parser. aug_transforms can only be used with_context=True

val_indices = get_val_indices(train_val_record_ids, record_ids_val)

def get_image_by_record_id(record_id):
    return get_image_path(combined_record_collection, record_id)

def get_mask_by_record_id(record_id):
    return record_to_mask(combined_record_collection, record_id)

coco_seg_dblock = DataBlock(
        blocks=(ImageBlock, MaskBlock(codes=data.class_list)), # ImageBlock is RGB by default, uses PIL
        get_x=get_image_by_record_id,
        splitter=IndexSplitter(val_indices),
        get_y=get_mask_by_record_id,
        # batch_tfms=batch_transfms,
        item_tfms = Resize(512),
        n_inp=1
    )

dset = coco_seg_dblock.datasets(source= record_ids_train)

We can grab a record of interest from the icevision record collection to inspect inference on a single result

In [ ]:
records_of_interest = []
for record in record_collection_with_negative_small_filtered_train:
    if "S1A_IW_GRDH_1SDV_20200724T020738_20200724T020804_033590_03E494_B457" in str(record.common.filepath):
        records_of_interest.append(record)

idx_of_interest = records_of_interest[6].common.record_id

idx = train_val_record_ids.index(idx_of_interest)

img, mask = dset[idx]

%matplotlib inline
skio.imshow(np.array(img)[:,:,0])
img = np.array(img)

In [ ]:
def normalize(img):
    img = img/255
    return img

Like inputs to icevision, we need to normalize.

In [ ]:
norm_arr = normalize(torch.Tensor(np.moveaxis(img, 2,0)))

In [ ]:
skio.imshow(norm_arr[0,:,:].cpu().detach().numpy())

And we can then run prediction like so

In [ ]:
pred_arr = model(norm_arr.unsqueeze(0))

In [ ]:
pred_arr.shape

In [ ]:
probs,classes = logits_to_classes(pred_arr)

dice_table = apply_conf_threshold(probs, classes, .4)

In [ ]:
apply_conf_threshold??

In [ ]:
dice_table.shape

The result from Fastai after confidence thresholding is a 2D array/

In [ ]:
skio.imshow(dice_table.detach().cpu().numpy())

In [ ]:
np.unique(dice_table.detach().cpu().numpy())

## Confusion Matrix Comparison for Unet and MaskRCNN

In this section we create and compare pixel-wise confusion matrices and instance-wise confusion matrices.

In [ ]:
from ceruleanml.evaluation import get_cm_for_torchscript_model_unet, get_cm_for_torchscript_model_mrcnn
from ceruleanml import data
from icevision.metrics.confusion_matrix import SimpleConfusionMatrix
# from icevision.models.checkpoint import model_from_checkpoint

We create a fastai dls from the dataset we created above.

In [ ]:
dls = coco_seg_dblock.dataloaders(source=train_val_record_ids, batch_size=1)

And then run inference and create the cm.

TODO Fastai Unet CM needs to be inspected, result shows no true labels for vessels

In [ ]:
cm_unet, f1_unet = get_cm_for_torchscript_model_unet(dls, model, fastai_unet_experiment_dir, semantic_mask_conf_thresh=.5, class_names=data.class_list, normalize = None, title="Fastai Unet Confusion Matrix: 29_Jun_2022_06_36_38")

In [ ]:
cm_unet, f1_unet = get_cm_for_torchscript_model_unet(dls, model, fastai_unet_experiment_dir, semantic_mask_conf_thresh=.5, class_names=data.class_list, normalize = "true", title="Fastai Unet Confusion Matrix: 29_Jun_2022_06_36_38")

The pixel wise mrcnn cm is correct. TODO this doesn't work with negative samples, only the instance confusion matrix does.

In [ ]:
get_cm_for_torchscript_model_mrcnn?

In [ ]:
_, test_tfms = get_tfms()
test_ds = Dataset(record_collection_test[:9], test_tfms)

In [ ]:
cm_mrcnn, f1_mrcnn = get_cm_for_torchscript_model_mrcnn(
    test_ds, 
    scripted_model, 
    save_path=icevision_experiment_dir, 
    mask_conf_threshold=.1, 
    bbox_conf_threshold=.1, 
    soft_dice_nms_threshold=.5, 
    normalize=None, 
    title=f"Torchvision MaskR-CNN Confusion Matrix: {icevision_experiment_dir}"
)

In [ ]:
cm_mrcnn, f1_mrcnn = get_cm_for_torchscript_model_mrcnn(
    valid_ds, scripted_model, save_path=icevision_experiment_dir, mask_conf_threshold=.01, bbox_conf_threshold=.7, normalize="true", class_names=data.class_list, title="Torchvision MaskR-CNN Confusion Matrix: 20_Jul_2022_00_14_15"
)


In [ ]:
checkpoint_path = '/root/data/experiments/cv2/20_Jul_2022_00_14_15_icevision_maskrcnn/state_dict_test_28_34_224_58.pt'

checkpoint_and_model = model_from_checkpoint(checkpoint_path, 
    model_name='torchvision.mask_rcnn', 
    backbone_name='resnet34_fpn',
    img_size=224, 
    classes=data.class_list,
    is_coco=False)

model = checkpoint_and_model["model"]
model_type = checkpoint_and_model["model_type"]
backbone = checkpoint_and_model["backbone"]
class_map = checkpoint_and_model["class_map"]
img_size = checkpoint_and_model["img_size"]
model_type, backbone, class_map, img_size

infer_dl = model_type.infer_dl(valid_ds, batch_size=1,shuffle=False)

preds = model_type.predict_from_dl(model, infer_dl, keep_images=True, detection_threshold=0.5)

In [ ]:
__all__ = ["COCOMetric", "COCOMetricType"]

from icevision.imports import *
from icevision.utils import *
from icevision.data import *
from icevision.metrics.metric import *


class COCOMetricType(Enum):
    """Available options for `COCOMetric`."""

    bbox = "bbox"
    mask = "segm"
    keypoint = "keypoints"


class COCOMetric(Metric):
    """Wrapper around [cocoapi evaluator](https://github.com/cocodataset/cocoapi)

    Calculates average precision.

    # Arguments
        metric_type: Dependent on the task you're solving.
        print_summary: If `True`, prints a table with statistics.
        show_pbar: If `True` shows pbar when preparing the data for evaluation.
    """

    def __init__(
        self,
        metric_type: COCOMetricType = COCOMetricType.bbox,
        iou_thresholds: Optional[Sequence[float]] = None,
        print_summary: bool = False,
        show_pbar: bool = False,
    ):
        self.metric_type = metric_type
        self.iou_thresholds = iou_thresholds
        self.print_summary = print_summary
        self.show_pbar = show_pbar
        self._records, self._preds = [], []

    def _reset(self):
        self._records.clear()
        self._preds.clear()

    def accumulate(self, preds):
        for pred in preds:
            self._records.append(pred.ground_truth)
            self._preds.append(pred.pred)

    def finalize(self) -> Dict[str, float]:
        with CaptureStdout():
            coco_eval = create_coco_eval(
                records=self._records,
                preds=self._preds,
                metric_type=self.metric_type.value,
                iou_thresholds=self.iou_thresholds,
                show_pbar=self.show_pbar,
            )
            coco_eval.evaluate()
            coco_eval.accumulate()

        with CaptureStdout(propagate_stdout=self.print_summary):
            coco_eval.summarize()

        stats = coco_eval.stats
        logs = {
            "AP (IoU=0.50:0.95) area=all": stats[0],
            "AP (IoU=0.50) area=all": stats[1],
            "AP (IoU=0.75) area=all": stats[2],
            "AP (IoU=0.50:0.95) area=small": stats[3],
            "AP (IoU=0.50:0.95) area=medium": stats[4],
            "AP (IoU=0.50:0.95) area=large": stats[5],
            "AR (IoU=0.50:0.95) area=all maxDets=1": stats[6],
            "AR (IoU=0.50:0.95) area=all maxDets=10": stats[7],
            "AR (IoU=0.50:0.95) area=all maxDets=100": stats[8],
            "AR (IoU=0.50:0.95) area=small maxDets=100": stats[9],
            "AR (IoU=0.50:0.95) area=medium maxDets=100": stats[10],
            "AR (IoU=0.50:0.95) area=large maxDets=100": stats[11],
        }

        self._reset()
        return logs


In [ ]:
preds[0].pred.detection

In [ ]:
preds[0].pred.detection.bboxes[0].xywh

In [ ]:
cm = SimpleConfusionMatrix() # this class is edited on SkyTruth fork
cm.accumulate(preds)

TODO highlight edits to icevision cm code and how to and where to install this edited icevision

In [ ]:
%matplotlib inline
_ = cm.finalize()

cm.plot(figsize=5, normalize=None)

In [ ]:
cm.class_map